In [1]:
#import pytest
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import pickle 
from sqlalchemy import create_engine, text

import os

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from time import sleep

In [2]:
my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
# Set up Chrome options
chrome_options = Options()
#chrome_options.add_argument("--headless")

# Set the custom User-Agent
chrome_options.add_argument(f"--user-agent={my_user_agent}")

In [3]:
user = "neondb_owner"
password = np.loadtxt("./neon_password.txt", dtype=str)
host = "ep-wild-night-a9bq078j-pooler.gwc.azure.neon.tech"
db = "xG"
engine = create_engine(f'postgresql://{user}:{password}@{host}/{db}')
conn = engine.connect()
query = text("SELECT count(*) FROM players")
exe = conn.execute(query) 

players = pd.read_sql("players", con=engine)
row_count = players.id.max()

print(row_count)

689


In [4]:
def get_team_quantities(driver):
    weight = 0
    height = 0
    nat2 = "NA"
    display_name = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[2]/h1/span").text
    if display_name[2] == ".":
        display_name = display_name[3:-10]
    else:
        display_name = display_name[2:-10]
    bio = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[5]/div/div/div[2]/div[1]/div").text
    biostr = bio.split("\n")
    for i in range(len(biostr)):
        if biostr[i] == "NACIONALIDADE":
            nat1 = biostr[i+1]
        elif biostr[i] == "DUPLA NACIONALIDADE":
            nat2 = biostr[i+1]
        elif biostr[i] == "NOME":
            full_name = biostr[i+1]
        elif biostr[i] == "POSIÇÃO":
            position = biostr[i+1]
        elif biostr[i] == "NASCIMENTO/IDADE":
            birth_year = biostr[i+1][:4]
        elif biostr[i] == "PÉ PREFERENCIAL":
            foot = biostr[i+1]
        elif biostr[i] == "ALTURA":
            height = biostr[i+1][:-3]
        elif biostr[i] == "PESO":
            weight = biostr[i+1][:-3]
            
    return np.array([display_name, birth_year, nat1, nat2, foot, 
                     position, full_name, height, weight])

In [5]:
league_url = "https://www.zerozero.pt/edicao/liga-portugal-betclic-2024-25/187713"


In [6]:
service = Service(executable_path="/usr/local/bin/chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(league_url)
sleep(1)

no_notifs = "/html/body/div[12]/div/div[3]/div/button[1]"
cookies = "/html/body/div[1]/div/div/div/div/div/div[2]/button[2]"

while len(driver.find_elements(By.XPATH, no_notifs)) == 0:
    sleep(0.1)
if len(driver.find_elements(By.XPATH, no_notifs)) != 0:
    button = driver.find_element(By.XPATH, no_notifs)
    button.click()
    sleep(0.5)
if len(driver.find_elements(By.XPATH, cookies)) != 0:
    button = driver.find_element(By.XPATH, cookies)
    button.click()
    sleep(0.5)


In [7]:
table_xpath = "/html/body/div[8]/div[3]/div[6]/div/div/div[1]/div[2]/table/tbody/tr/td/div[1]/div/table/tbody"
body = driver.find_element(By.XPATH, table_xpath)

rows = body.find_elements(By.TAG_NAME, "tr")
team_urls = [ rows[i].find_elements(By.TAG_NAME, "a")[1].get_attribute("href") for i in range(len(rows)) ]

driver.close()

In [8]:
team_urls[16:17]

['https://www.zerozero.pt/equipa/farense/10?epoca_id=154']

In [9]:
quants = []
#team_url = team_urls[1]
#if True:
for team_url in team_urls[16:17]:

    service = Service(executable_path="/usr/local/bin/chromedriver-linux64/chromedriver")
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(team_url)
    sleep(1)

    no_notifs = "/html/body/div[12]/div/div[3]/div/button[1]"
    cookies = "/html/body/div[1]/div/div/div/div/div/div[2]/button[2]"
    while len(driver.find_elements(By.XPATH, no_notifs)) == 0:
        sleep(0.1)
    if len(driver.find_elements(By.XPATH, no_notifs)) != 0:
        button = driver.find_element(By.XPATH, no_notifs)
        button.click()
        sleep(0.5)
    if len(driver.find_elements(By.XPATH, cookies)) != 0:
        button = driver.find_element(By.XPATH, cookies)
        button.click()
        sleep(0.5)

    squad = driver.find_element(By.ID, "team_squad")
    players = squad.find_elements(By.CLASS_NAME, "staff")
    #player = players[0]
    for player in players:
    #if True:
        player.find_element(By.CLASS_NAME, "text").click()
        cnt = 0
        while len(driver.find_elements(By.XPATH, no_notifs)) == 0 and cnt < 5:
            cnt += 0.1
            sleep(0.1)
        if len(driver.find_elements(By.XPATH, no_notifs)) != 0:
            button = driver.find_element(By.XPATH, no_notifs)
            button.click()
            sleep(0.5)
        sleep(1)
    
        quants.append(get_team_quantities(driver))
        driver.back()

        
quants = np.array(quants)

driver.close()

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=139.0.7258.154); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5bd78a3fd1da <unknown>
#1 0x5bd789ea3780 <unknown>
#2 0x5bd789eb6dbb <unknown>
#3 0x5bd789eb5b72 <unknown>
#4 0x5bd789eaacd9 <unknown>
#5 0x5bd789ea8f1f <unknown>
#6 0x5bd789eacc58 <unknown>
#7 0x5bd789eacce3 <unknown>
#8 0x5bd789ef4d25 <unknown>
#9 0x5bd789ef54e1 <unknown>
#10 0x5bd789ee8f76 <unknown>
#11 0x5bd789f1adcd <unknown>
#12 0x5bd789ee8e6a <unknown>
#13 0x5bd789f1af6e <unknown>
#14 0x5bd789f40bfc <unknown>
#15 0x5bd789f1ab73 <unknown>
#16 0x5bd789ee76e3 <unknown>
#17 0x5bd789ee8331 <unknown>
#18 0x5bd78a3c302b <unknown>
#19 0x5bd78a3c6e1e <unknown>
#20 0x5bd78a3a9f99 <unknown>
#21 0x5bd78a3c79e8 <unknown>
#22 0x5bd78a38e92f <unknown>
#23 0x5bd78a3ea788 <unknown>
#24 0x5bd78a3ea966

In [ ]:
#quants = np.array(quants)
#quants = quants[:-41,:]
quants

array([['', '2004', 'Brasil', 'NA', 'Esquerdo', 'Guarda Redes',
        'Lucas Moura Alviano', '187', '83'],
       ['Guillermo Ochoa', '1985', 'México', 'Espanha', 'Direito',
        'Guarda Redes', 'Francisco Guillermo Ochoa Magaña', '183', '78'],
       ['Pedro Trigueira', '1988', 'Portugal', 'NA', 'Direito',
        'Guarda Redes', 'Pedro José da Silva Trigueira', '192', '88'],
       ['Simão Bertelli', '1993', 'Brasil', 'NA', 'Direito',
        'Guarda Redes', 'Simão Verza Bertelli', '187', '89'],
       ['Fernando Fonseca', '1997', 'Portugal', 'NA', 'Direito',
        'Defesa (Defesa Direito)', 'Fernando Manuel Ferreira Fonseca',
        '182', '66'],
       ['', '1999', 'Brasil', 'NA', 'Direito', 'Defesa (Defesa Direito)',
        'Leandro Santos do Nascimento', '183', '75'],
       ['Tomás Tavares', '2001', 'Portugal', 'Cabo Verde', 'Direito',
        'Defesa (Defesa Direito)', 'Tomás Franco Tavares', '184', '69'],
       ['Nacho Rodríguez', '2003', 'Uruguai', 'NA', 'Direito',


In [ ]:
#quants = np.array(quants)[:-1, :]
dict = {
    "id": np.arange(0, len(quants)) + 1 + row_count,
    "display_name": quants[:, 0],
    "birth_year": quants[:, 1],
    "nationality": quants[:, 2],
    "second_nationality": quants[:, 3],
    "prefered_foot": quants[:, 4],
    "position": quants[:, 5],
    "full_name": quants[:, 6],
    "height": quants[:, 7],
    "weight": quants[:, 8]
}

In [ ]:
dict["display_name"]

array(['', 'Guillermo Ochoa', 'Pedro Trigueira', 'Simão Bertelli',
       'Fernando Fonseca', '', 'Tomás Tavares', 'Nacho Rodríguez',
       'Jorge Teixeira', 'Thia', 'Aderllan Santos', 'Clayt',
       'Cristian Devenish', 'Baptiste Roux', 'Kiki Afonso', 'Eric Veiga',
       'Rafa Rodrigues', 'Zé Ricardo', 'Gustavo Assunção', 'Jaume Grau',
       'Tiago Galletto', 'Gustavo Mendonça', 'Jon', 'Lucas Piazón', 'cas',
       'Luís Silva', 'Giorgi', 'Vasco Lopes', 'Yair Mena',
       'Babatunde Akinsola', 'Samu', 'Issiaka Kamate', 'Zé Luís', 'Nenê',
       'Rodrigo Ribeiro', 'John Mercado'], dtype='<U56')

In [ ]:
with open("players.pkl", "wb") as handle:    
    pickle.dump(dict, handle,
                protocol=pickle.HIGHEST_PROTOCOL) 